In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-03-20 06:18:24.211387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 06:18:24.211452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 06:18:24.214228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 06:18:24.232469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 06:18:26.877901: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels

# Get list of image paths
image_paths = list(paths.list_images("../HAM10000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/10015
[INFO]: Processed 2000/10015
[INFO]: Processed 3000/10015
[INFO]: Processed 4000/10015
[INFO]: Processed 5000/10015
[INFO]: Processed 6000/10015
[INFO]: Processed 7000/10015
[INFO]: Processed 8000/10015
[INFO]: Processed 9000/10015
[INFO]: Processed 10000/10015
(10015, 224, 224, 3)
(10015,)


In [8]:
num_classes = len(np.unique(labels))

In [9]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        data = data.reshape(data.shape[0],data.shape[1] * data.shape[2] * data.shape[3])
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [ ]:
from sklearn.linear_model import LogisticRegression
# LR
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)


[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6956087824351297
Sensitivity: 0.25363529073550395
Specificity: 0.8962973823522512
AUC-ROC: 0.841433503882206
MCC: 0.31332139115520646
Precision: 0.37464968589323433
F1 Score: 0.276435237823592

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6776447105788423
Sensitivity: 0.2626050892575413
Specificity: 0.8918742029987221
AUC-ROC: 0.8426221039883266
MCC: 0.27708334754477243
Precision: 0.3891676613065778
F1 Score: 0.28618715811544565

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.7015968063872255
Sensitivity: 0.2668290382638809
Specificity: 0.9002908755220119
AUC-ROC: 0.8193406183574385
MCC: 0.33383628906974616
Precision: 0.42548920759517744
F1 Score: 0.2820852713281914

[INFO] Fold 5 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6926147704590818
Sensitivity: 0.271998143519703
Specificity: 0.8951586389520092
AUC-ROC: 0.8561678242391423
MCC: 0.30780211792004236
Precision: 0.5925351854353699
F1 Score: 0.301626688391526

[INFO] Fold 6 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6963036963036963
Sensitivity: 0.24544516052226822
Specificity: 0.893255135720761
AUC-ROC: 0.8241732917009529
MCC: 0.30019969787392947
Precision: 0.37762265291352864
F1 Score: 0.26315573192408376

[INFO] Fold 7 / 10 for LR


In [ ]:
# SVM
#svm_model = SVC(probability=True)
#evaluate_model(svm_model, 'SVM', data, labels)


#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)

In [ ]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)



In [ ]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)


In [ ]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)

In [ ]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the VGG16 model pre-trained on ImageNet data
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Build a new model using VGG16 as a feature extractor
with tf.device('/GPU:0'):
    model_vgg16 = Sequential()
    model_vgg16.add(vgg16_model)
    model_vgg16.add(Flatten())
    model_vgg16.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_vgg16.add(Dropout(0.5, seed = 0))
    model_vgg16.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_vgg16, 'VGG16', data, labels)

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the EfficientNetB7 model pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Build a new model using EfficientNetB7 as a feature extractor
with tf.device('/GPU:0'):
    model_efficientnet = Sequential()
    model_efficientnet.add(efficientnet_model)
    model_efficientnet.add(Flatten())
    model_efficientnet.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_efficientnet.add(Dropout(0.5, seed=0))
    model_efficientnet.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_efficientnet, 'EfficientNetB0', data, labels)

In [ ]:
from tensorflow.keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the ResNet50 model pre-trained on ImageNet data
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build a new model using ResNet50 as a feature extractor
with tf.device('/GPU:0'):
    model_resnet50 = Sequential()
    model_resnet50.add(resnet50_model)
    model_resnet50.add(Flatten())
    model_resnet50.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_resnet50.add(Dropout(0.5, seed=0))
    model_resnet50.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_resnet50, 'ResNet50', data, labels)


In [ ]:
# Save results to a CSV file
results_df.to_csv('DS7_baseline.csv', index=False)

In [ ]:
results_df